# Voici un programme pour tester notre modèle 

In [153]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [154]:
#la lecture des données
train=pd.read_csv("train.csv")

In [155]:
#Mappage des données de la colonne Age
train['Age'] = train['Age'].fillna(train.query('Pclass==1')['Age'].median())
train['Age'] = train['Age'].fillna(train.query('Pclass==2')['Age'].median())
train['Age'] = train['Age'].fillna(train.query('Pclass==3')['Age'].median())

#Mappage des données de la colonne Sex
train.loc[train.Sex=="male",'Sex']='0'
train.loc[train.Sex=="female",'Sex']='1'

#Mappage des données de la colonne Embarked
train.loc[train.Embarked=="C",'Embarked']='1'
train.loc[train.Embarked=="S",'Embarked']='2'
train.loc[train.Embarked.isna(),'Embarked']='2'
train.loc[train.Embarked=="Q",'Embarked']='3'

train['Fare'] = train['Fare'].fillna(train['Fare'].median())
train['Fare'] =(train['Fare']-train['Fare'].mean())/train['Fare'].std()
train['Age']=(train['Age']-train['Age'].mean())/train['Age'].std()

features_drop=['Name','Ticket','Cabin','PassengerId']
train=train.drop(features_drop, axis=1)

## On utilise des nouveaux passagers pour tester le modèle

In [156]:
#Voici des nouveaux passagers intéressants avec des données personnaliseés
nouveaux_passagers = pd.DataFrame([
    #Passager numéro 1
    {'PassengerId':1,
    "Pclass": 1,
    "Name": "Smith, Mr.John",
    "Sex": "male",
    "Age": 25,
    "SibSp": 2,
    "Parch": 1,
    "Ticket": "PC 17582",
    "Fare": 70,
    "Cabin": "C62 C64",
    "Embarked": "C"},
    
    #Passager numéro 2
    {'PassengerId':2,
    "Pclass": 1,
    "Name": "Lin,Mr. Hongxiang",
    "Sex": "male",
    "Age": 20,
    "SibSp": 1,
    "Parch": 3,
    "Ticket": "PC 15532",
    "Fare": 500,
    "Cabin": "B43 B78",
    "Embarked": "C"},
    
    #Passager numéro 3
    {'PassengerId':3,
    "Pclass": 3,
    "Name": "Antoine, Mr.David",
    "Sex": "male",
    "Age": 45,
    "SibSp": 4,
    "Parch": 6,
    "Ticket": "345678",
    "Fare": 8,
    "Cabin": np.nan,
    "Embarked": "Q"},
    
    #Passager numéro 4
    {'PassengerId':4,
    "Pclass": 2,
    "Name": "Alice, Mlle.Sarah",
    "Sex": "female",
    "Age": 28,
    "SibSp": 0,
    "Parch": 1,
    "Ticket": "23444",
    "Fare": 40,
    "Cabin": np.nan,
    "Embarked": "S"}
])
nouveaux_passagers.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,"Smith, Mr.John",male,25,2,1,PC 17582,70,C62 C64,C
1,2,1,"Lin,Mr. Hongxiang",male,20,1,3,PC 15532,500,B43 B78,C
2,3,3,"Antoine, Mr.David",male,45,4,6,345678,8,NaN,Q
3,4,2,"Alice, Mlle.Sarah",female,28,0,1,23444,40,NaN,S


# L'algorithme de prédiction avec Random Forest


In [157]:
#Importation du modèle de Random Forest que nous allons utiliser car il est le meilleur modèle selon notre score de submission sur Kaggle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [158]:
#On utilise les données de ces passagers comme un nouveau test
test = nouveaux_passagers
test_ids=test['PassengerId']

In [159]:
#Nous allons reproduire les memes modifications faites sur la dataframe "train" sur ce dataframe pour pouvoir le tester.

#Age
test['Age'] = test['Age'].fillna(test['Age'].median())
test['Age']=(test['Age']-test['Age'].mean())/test['Age'].std()

#Sex
test.loc[test.Sex=="male",'Sex']='0'
test.loc[test.Sex=="female",'Sex']='1'

#Embarked
test.loc[test.Embarked=="C",'Embarked']='1'
test.loc[test.Embarked=="S",'Embarked']='2'
test.loc[test.Embarked.isna(),'Embarked']='2'
test.loc[test.Embarked=="Q",'Embarked']='3'

#Fare
test['Fare'] = test['Fare'].fillna(test['Fare'].median())
test['Fare']=(test['Fare']-test['Fare'].mean())/test['Fare'].std()

#Features moins importants
features_drop=['Name','Ticket','Cabin','PassengerId']
test=test.drop(features_drop, axis=1)

In [160]:
#Les variables d'entrées sont toutes les colonnes de la base de donnée, et la variable de sortie est "Survived"
X=train.drop('Survived', axis=1)
y=train["Survived"]

In [161]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

In [162]:
model=RandomForestClassifier(n_estimators=160,max_depth=13)

In [163]:
model.fit(X_train,y_train)

RandomForestClassifier(max_depth=13, n_estimators=160)

In [164]:
#On prédit sur le dataframe "test"
submission_preds=model.predict(test)

In [165]:
#Creation du dataframe resultat qui contient uniquement deux colonnes, "PassengerId" et "Survived"(prédit auaparavant)
resultat=pd.DataFrame({"PassengerId":test_ids.values,
                 "Survived":submission_preds,
                })
#Voici les résultats de survie par la prédiction du Ramdom Forest
resultat.head()

,PassengerId,Survived
0,1,0
1,2,1
2,3,0
3,4,1


## On peut également générer un fichier test_submission.csv

In [166]:
resultat.to_csv("test_submission.csv", index=False)